기상자료개방포털 사이트에서는 API 서비스를 운영하고 있다  
이를 이용해서 현재 기상 데이터를 가져와 다음날 평균기온을 추론하는 코드를 작성해보려한다

In [2]:
import requests
from urllib import parse
import tensorflow as tf
import keras
from tensorflow.keras.models import load_model 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime

api를 통해 데이터를 불러온다  
참고 글 :  https://jvvp.tistory.com/1035

In [73]:
# 데이터를 불러올 날짜 설정  
# 현재는 어제 날짜로 설정  
now = datetime.datetime.now() - datetime.timedelta(1)
now_str = str(now.year)+str(now.month).zfill(2)+str(now.day)

In [61]:
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
key = 'cT4JNsYj6lxarKAfoCWMZH4YY4mzzyc7kS3%2BNb1CYJVFh614%2FO6qMi%2FLZTSmCVHXkY7%2B3JsSdOClQ8RtBE9APQ%3D%3D'



param = f'?{parse.quote_plus("ServiceKey")}={key}&' + parse.urlencode({
    parse.quote_plus('numOfRows') : '10',
    parse.quote_plus('pageNo') : '1',
    parse.quote_plus('dataCd') : 'ASOS',
    parse.quote_plus('dateCd') : 'DAY',
    parse.quote_plus('startDt') : now_str,
    parse.quote_plus('endDt') : now_str,
    parse.quote_plus('stnIds') : '108'})

response = requests.get(url+param)
print(response.text)

soup = BeautifulSoup(response.content, 'lxml')

<?xml version="1.0" encoding="UTF-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><stnId>108</stnId><stnNm>서울</stnNm><tm>2021-01-09</tm><avgTa>-12.2</avgTa><minTa>-16.6</minTa><minTaHrmt>614</minTaHrmt><maxTa>-7.5</maxTa><maxTaHrmt>1602</maxTaHrmt><mi10MaxRnHrmt></mi10MaxRnHrmt><hr1MaxRn></hr1MaxRn><hr1MaxRnHrmt></hr1MaxRnHrmt><sumRn></sumRn><maxInsWs>11.4</maxInsWs><maxInsWsWd>270</maxInsWsWd><maxInsWsHrmt>1644</maxInsWsHrmt><maxWs>6.5</maxWs><maxWsWd>290</maxWsWd><maxWsHrmt>1628</maxWsHrmt><avgWs>2.6</avgWs><hr24SumRws>2221</hr24SumRws><maxWd>270</maxWd><avgTd>-21.5</avgTd><minRhm>35</minRhm><minRhmHrmt>1207</minRhmHrmt><avgRhm>46.3</avgRhm><avgPv>1.1</avgPv><avgPa>1014.0</avgPa><maxPs>1027.4</maxPs><maxPsHrmt>2222</maxPsHrmt><minPs>1023.3</minPs><minPsHrmt>1441</minPsHrmt><avgPs>1025.4</avgPs><ssDur>9.7</ssDur><sumSsHr>9.0</sumSsHr><hr1MaxIcsrHrmt>1200</hr1MaxIcsrHrmt><hr1MaxIcsr>1.99</hr1MaxI

이렇게 api로 호출한 데이터를 데이터프레임 형태로 정리해보자  
아래 태그명은 공공데이터포털사이트에서 제공해주는 활용가이드를 사용하여 정리하였다  

In [62]:
col_raw = 'stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRnHrmt,hr1MaxRn,hr1MaxRnHrmt,sumRn,maxInsWs,maxInsWsWd,maxInsWsHrmt,maxWs,maxWsWd,maxWsHrmt,avgWs,hr24SumRws,maxWd,avgTd,minRhm,minRhmHrmt,avgRhm,avgPv,avgPa,maxPs,maxPsHrmt,minPs,minPsHrmt,avgPs,ssDur,sumSsHr,hr1MaxIcsrHrmt,hr1MaxIcsr,sumGsr,ddMefs,ddMefsHrmt,ddMes,ddMesHrmt,sumDpthFhsc,avgTca,avgLmac,avgTs,minTg,avgCm5Te,avgCm10Te,avgCm20Te,avgCm30Te,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur'
col_raw = col_raw.lower()
col=col_raw.split(',')

In [63]:
data = []
cnt=0
for c in col:
    data.append([i.text for i in soup.select('item '+c)])

학습데이터와 같은 방식으로 전처리하기 위해 동일한 컬럼명으로 바꿔준다.  
'강수 계속시간(hr)','10분 최다 강수량(mm)' 이 2가지 값은 csv로 다운받았을 때는 포함되어 있었지만 api로 불러왔을 때는 포함되지 않았다  

In [64]:
col_name = ['지점', '지점명', '일시', '평균기온(°C)', '최저기온(°C)', '최저기온 시각(hhmi)', '최고기온(°C)',
            '최고기온 시각(hhmi)', '10분 최다강수량 시각(hhmi)','1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)',
            '일강수량(mm)', '최대 순간 풍속(m/s)','최대 순간 풍속 풍향(16방위)', '최대 순간풍속 시각(hhmi)', '최대 풍속(m/s)',
            '최대 풍속 풍향(16방위)','최대 풍속 시각(hhmi)', '평균 풍속(m/s)', '풍정합(100m)', '최다풍향(16방위)',
            '평균 이슬점온도(°C)', '최소 상대습도(%)', '최소 상대습도 시각(hhmi)', '평균 상대습도(%)','평균 증기압(hPa)',
            '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최고 해면기압 시각(hhmi)','최저 해면기압(hPa)', '최저 해면기압 시각(hhmi)',
            '평균 해면기압(hPa)', '가조시간(hr)','합계 일조시간(hr)', '1시간 최다일사 시각(hhmi)', '1시간 최다일사량(MJ/m2)',
            '합계 일사량(MJ/m2)','일 최심신적설(cm)', '일 최심신적설 시각(hhmi)', '일 최심적설(cm)', '일 최심적설 시각(hhmi)',
            '합계 3시간 신적설(cm)', '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)','최저 초상온도(°C)',
            '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)','평균 20cm 지중온도(°C)', '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)',
            '1.0m 지중온도(°C)', '1.5m 지중온도(°C)', '3.0m 지중온도(°C)', '5.0m 지중온도(°C)',
            '합계 대형증발량(mm)', '합계 소형증발량(mm)', '9-9강수(mm)', '기사', '안개 계속시간(hr)']

In [65]:
DF = pd.DataFrame(data).T
DF.columns = col_name
DF

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),10분 최다강수량 시각(hhmi),1시간 최다강수량(mm),...,0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),기사,안개 계속시간(hr)
0,108,서울,2021-01-09,-12.2,-16.6,614,-7.5,1602,,,...,1.5,5.6,8.8,15.2,17.2,0.9,1.3,,,


#### 학습에 사용한 변수  
'평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '1시간 최다강수량(mm)', '일강수량(mm)',  
'최대 순간 풍속(m/s)', '최대 순간 풍속 풍향(16방위)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)',  
'평균 풍속(m/s)', '풍정합(100m)', '최다풍향(16방위)', '평균 이슬점온도(°C)', '최소 상대습도(%)',  
'평균 상대습도(%)', '평균 증기압(hPa)', '평균 현지기압(hPa)', '최고 해면기압(hPa)',  
'최저 해면기압(hPa)', '평균 해면기압(hPa)', '가조시간(hr)', '합계 일조시간(hr)',  
'1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)', '일 최심신적설(cm)', '일 최심적설(cm)',  
'합계 3시간 신적설(cm)', '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)',  
'최저 초상온도(°C)', '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)',  
'평균 20cm 지중온도(°C)', '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)',  
'1.0m 지중온도(°C)', '1.5m 지중온도(°C)', '3.0m 지중온도(°C)', '5.0m 지중온도(°C)',  
'합계 대형증발량(mm)', '합계 소형증발량(mm)', '9-9강수(mm)', '안개 계속시간(hr)'

In [66]:
train_col = ['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '1시간 최다강수량(mm)', '일강수량(mm)',
       '최대 순간 풍속(m/s)', '최대 순간 풍속 풍향(16방위)', '최대 풍속(m/s)', '최대 풍속 풍향(16방위)',
       '평균 풍속(m/s)', '풍정합(100m)', '최다풍향(16방위)', '평균 이슬점온도(°C)', '최소 상대습도(%)',
       '평균 상대습도(%)', '평균 증기압(hPa)', '평균 현지기압(hPa)', '최고 해면기압(hPa)',
       '최저 해면기압(hPa)', '평균 해면기압(hPa)', '가조시간(hr)', '합계 일조시간(hr)',
       '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)', '일 최심신적설(cm)', '일 최심적설(cm)',
       '합계 3시간 신적설(cm)', '평균 전운량(1/10)', '평균 중하층운량(1/10)', '평균 지면온도(°C)',
       '최저 초상온도(°C)', '평균 5cm 지중온도(°C)', '평균 10cm 지중온도(°C)',
       '평균 20cm 지중온도(°C)', '평균 30cm 지중온도(°C)', '0.5m 지중온도(°C)',
       '1.0m 지중온도(°C)', '1.5m 지중온도(°C)', '3.0m 지중온도(°C)', '5.0m 지중온도(°C)',
       '합계 대형증발량(mm)', '합계 소형증발량(mm)', '9-9강수(mm)', '안개 계속시간(hr)']

In [67]:
DF = DF[train_col].replace('', np.nan)

In [68]:
DF.isna().sum()

평균기온(°C)             0
최저기온(°C)             0
최고기온(°C)             0
1시간 최다강수량(mm)        1
일강수량(mm)             1
최대 순간 풍속(m/s)        0
최대 순간 풍속 풍향(16방위)    0
최대 풍속(m/s)           0
최대 풍속 풍향(16방위)       0
평균 풍속(m/s)           0
풍정합(100m)            0
최다풍향(16방위)           0
평균 이슬점온도(°C)         0
최소 상대습도(%)           0
평균 상대습도(%)           0
평균 증기압(hPa)          0
평균 현지기압(hPa)         0
최고 해면기압(hPa)         0
최저 해면기압(hPa)         0
평균 해면기압(hPa)         0
가조시간(hr)             0
합계 일조시간(hr)          0
1시간 최다일사량(MJ/m2)     0
합계 일사량(MJ/m2)        0
일 최심신적설(cm)          1
일 최심적설(cm)           0
합계 3시간 신적설(cm)       1
평균 전운량(1/10)         0
평균 중하층운량(1/10)       0
평균 지면온도(°C)          0
최저 초상온도(°C)          0
평균 5cm 지중온도(°C)      0
평균 10cm 지중온도(°C)     0
평균 20cm 지중온도(°C)     0
평균 30cm 지중온도(°C)     0
0.5m 지중온도(°C)        0
1.0m 지중온도(°C)        0
1.5m 지중온도(°C)        0
3.0m 지중온도(°C)        0
5.0m 지중온도(°C)        0
합계 대형증발량(mm)         0
합계 소형증발량(mm)         0
9-9강수(mm)            1
안개 계속시간(hr)

학습 데이터와 같은 방식으로 전처리한다.  

In [69]:
rain_col = [i for i in train_col if '강수' in i]
snow_col = [i for i in train_col if '적설' in i]

DF[rain_col+snow_col] = DF[rain_col+snow_col].fillna(0)
DF[['안개 계속시간(hr)','합계 대형증발량(mm)','합계 소형증발량(mm)']] = DF[['안개 계속시간(hr)','합계 대형증발량(mm)','합계 소형증발량(mm)']].fillna(0)
DF = DF.astype('float')

In [70]:
DF

,평균기온(°C),최저기온(°C),최고기온(°C),1시간 최다강수량(mm),일강수량(mm),최대 순간 풍속(m/s),최대 순간 풍속 풍향(16방위),최대 풍속(m/s),최대 풍속 풍향(16방위),평균 풍속(m/s),...,평균 30cm 지중온도(°C),0.5m 지중온도(°C),1.0m 지중온도(°C),1.5m 지중온도(°C),3.0m 지중온도(°C),5.0m 지중온도(°C),합계 대형증발량(mm),합계 소형증발량(mm),9-9강수(mm),안개 계속시간(hr)
0,-12.2,-16.6,-7.5,0.0,0.0,11.4,270.0,6.5,290.0,2.6,...,-0.3,1.5,5.6,8.8,15.2,17.2,0.9,1.3,0.0,0.0


과거데이터로 학습시킨 모델을 불러온다.

In [71]:
model = load_model('temp_1day_prediction.h5')

In [72]:
print('내일 예상 평균 기온 = ', round(model.predict(DF.values)[0][0],2),'(°C)')

내일 예상 평균 기온 =  -7.74 (°C)
